In [ ]:
!pip install -q pandas scikit-learn matplotlib datasets transformers wandb

In [ ]:
TRACK = True

In [ ]:
import wandb
if TRACK:
    wandb.login()

In [ ]:
from datasets import load_dataset

train = load_dataset("ai2_arc", 'ARC-Easy', split='train').to_pandas()
test = load_dataset("ai2_arc", 'ARC-Easy', split='test').to_pandas()
dev = load_dataset("ai2_arc", 'ARC-Easy', split='validation').to_pandas()

In [ ]:
import torch
from transformers import AutoTokenizer, BertConfig, BertForMultipleChoice
import random
import os
import numpy as np

SEED = 2023
def seed_everything(seed=2023):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Model is in device: {device}')

# config (BertConfig) — Model configuration class with all the parameters of the model. Initializing with a config file does not load the weights associated with the model, only the configuration.
configuration = BertConfig()
model = BertForMultipleChoice(configuration).to(device)

#model_name = 'bert-base-uncased'
model_name= 'dmis-lab/biobert-base-cased-v1.1-squad'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from multiple_choice_processor import MultipleChoiceProcessor

BATCH_SIZE = 16

processor = MultipleChoiceProcessor(tokenizer, train, dev, test)
train_loader, val_loader, test_loader = processor.create_datasets(batch_size=BATCH_SIZE, train_batch_size=BATCH_SIZE)

In [ ]:
from train_eval import TrainingPipeline
from torch.optim import AdamW
from torch.optim.lr_scheduler import CyclicLR

EPOCHS = 6
LR = 0.0001

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)
scheduler = CyclicLR(optimizer, base_lr=LR/10, max_lr=LR, cycle_momentum=False, mode='triangular2')

training_pipeline = TrainingPipeline(model, device, train_loader, val_loader, optimizer=optimizer, scheduler=scheduler, track=TRACK, num_epochs=EPOCHS, lr=LR, model_checkpoint=True)

train_loss_list, train_accuracy_list, val_loss_list, val_accuracy_list = training_pipeline.train()

In [ ]:
test_loss, test_accuracy = training_pipeline.evaluate()